In [7]:
import pandas as pd
import pandas_datareader as pdr
import seaborn as sns
import numpy as np
import yfinance as yf
import datetime
import requests
import os, sys
import tqdm
import pickle
import glob
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy.optimize import minimize
from nsepy import *

# Collect Data

In [ ]:
""" Collect all stock data from ../dat_eodhd_dow30/"""

# Set the path
path = '../dat_eodhd_dow30/'
all_files = glob.glob(os.path.join(path, "*.pkl"))

# Create an empty list to hold DataFrames
dfs = []

# Iterate through all files and append the adjusted_close column to the dfs list
for filename in all_files:
    # Read the .pkl file into a DataFrame
    df = pd.read_pickle(filename)
    
    # Extract the adjusted_close column
    adj_close = df[['adjusted_close']]
    
    # Rename the column with the filename
    adj_close.columns = [os.path.basename(filename).replace('.pkl', '')]
    
    # Append the adjusted_close DataFrame to the dfs list
    dfs.append(adj_close)

# Concatenate all DataFrames along the columns axis
final_df = pd.concat(dfs, axis=1)

# EDA

In [ ]:
""" step 1: plot daily price of all the stocks """

plt.figure()
final_df.plot(subplots = True,figsize = (15,120))
# plt.title("Close prices of all stocks")
plt.show()

In [ ]:
""" step 2: finding dailt returns"""

daily_returns = final_df/final_df.shift(1)
daily_returns

In [ ]:
""" step 3: In finance, we generally use logarithmic returns for analysis"""

log_returns = np.log(final_df/final_df.shift(1))
log_returns

In [ ]:
log_returns.mean() # for daily return
log_returns.mean() * 252 # stock market is open for 252 days in a year

In [ ]:
# Max and Min Retrun
max_return = log_returns.mean().max() * 252
min_return = log_returns.mean().min() * 252

# Find the best and worst ticker (maximum return)
best_ticker = log_returns.mean().idxmax()
worst_ticker = log_returns.mean().idxmin()

print(f"{best_ticker}: {max_return * 100:.4f}%")
print(f"{worst_ticker}: {min_return * 100:.4f}%")

In [ ]:
log_returns.std() # standard deviation
log_returns.cov() # covariance
log_returns.corr() # correlations

In [ ]:
# Create the heatmap with annotations
plt.figure(figsize=(15,8))
sns.heatmap(log_returns.corr(),linecolor='white',linewidths=1,annot=True)
plt.title("correlation heatmap of stocks")
plt.show()

In [ ]:
# Create the heatmap without annotations
plt.figure(figsize=(15,8))
sns.heatmap(log_returns.corr(), linecolor='white', linewidths=1, annot=False)
plt.title("Correlation heatmap of stocks")
plt.show()

### Using heatmap to find strongly positive or strongly negative correlated stocks

In [ ]:
# Create the heatmap with annotations
plt.figure(figsize=(15,8))
c = log_returns.corr()
sns.heatmap(((c > 0.68) | (c < -0.68)) & (c != 1),linecolor='white',linewidths=1,annot = True)
plt.show()

In [ ]:
# Create the heatmap without annotations
plt.figure(figsize=(15,8))
c = log_returns.corr()
sns.heatmap(((c > 0.68) | (c < -0.68)) & (c != 1), linecolor='white', linewidths=1, annot=False)
plt.show()

- From the heatmaps, we can observe that there is no pair with negative correlation
- We only have some pairs of stocks from banking sector with  high positive correlation

In [ ]:
sns.pairplot(final_df,palette='coolwarm')
plt.show()

- The pair plots also signify the same result that there is no pair of stocks with high negative correlation. We don't find any pair-plot with upper-left to lower-right pattern.
- The pairs with high positive correlation have scatter plot with lower-left to upper-right pattern .
- Other pairs don't form any pattern.

# Markowitz Model
- We model our assets by their expected return, $E[R]$ and their risk, which is expressed as their standard deviation, $\sigma$
- Our investment decisions are expressed by investing 100% of our wealth in assets( here, stocks), where each particular investment represents a proportion of our total wealth. 
- We will now implement Markowitz Model. This model assists in the selection of the most efficient portfolios by analyzing various possible portfolios of the selected stocks.
- We invest $w_i$ in $stock_i$ for every i, such that

 $$\Sigma^{n}_{i=1} w_i = 1$$
 - The expected return of the portfolio constructed would be

$$E[R_p] = \Sigma^{n}_{i=1} w_i E[R_i]$$
- the risk associated with the portfolio would be

$$\sigma^2(R_p) = \Sigma^{n}_{i=1} w_i^2 \sigma^2(R_i) + \Sigma^{}_{i=1}\Sigma^{}_{j {\neq} i} w_i w_j \sigma(R_i) \sigma(R_j) \rho_{ij}$$

- $E[R_i]$ is the annual expected return of $i$th stock, $\sigma(R_i)$ corrsponds to annual standard deviation of $i$th stock and $\rho_{ij}$ is the correlation between the logarithmic returns $i$th and the $j$th stock.
- An efficient portfolio is one that maximizes return for a given level of risk. Our task is to select adequate weights $w_i$ to get the efficient portfolio

## Implementation
-  Let $W_{1 \times n}$ be a array containing the weights $w_i$ such that $\Sigma^{n}_{i=1} w_i = 1$ and $E[R]_{ n\times 1}$ be another array containing annual expected returns of n stocks present in the portfolio and $C$ be the covariance matrix of annual returns of  stocks, then

$$E[R_p] = WE[R]$$

$$ \sigma^2(R_p) = W^TCW $$

### Sharpe Ratio

- It is a statistical measure used in Modern Portfolio Theory.
- The Sharpe ratio measures the performance of an investment compared to a risk-free asset, after adjusting for its risk. It is defined as the difference between the returns of the investment and the risk-free return, divided by the standard deviation of the investment.
- A portfolio with a higher Sharpe ratio is considered to have best risk-adjusted returns.

$$ S = \frac{E[R_p] - R_f}{\sigma(R_p)} $$

Here, $R_f$ is the risk free rate of return. I have taken risk free rate as 10 year government bond rate of India on July 9th, i.e 6.19%

In [ ]:
# A function for generating a numpy array containing random weights that add upto 1
def RandWeights(size):
    weight = np.random.dirichlet(np.ones(size))
    return weight

In [ ]:
risk_free_rate = 0.0619 # 6.19% on July 9

# A function to generate the avg return, risk and the sharpe ratio of the portfolio 
# correponding to the weight array passed
def portfolio_stats(weight):

    # Convert to array in case list was passed instead.
    weight = np.array(weight)
    port_return = np.sum(log_returns.mean() * weight) * 250
    port_risk = np.sqrt(np.dot(weight.T, np.dot(log_returns.cov() * 250, weight)))
    sharpe = (port_return - risk_free_rate)/port_risk

    return {'return': port_return, 'risk': port_risk, 'sharpe': sharpe}

In [ ]:
# Trying to generate random weights

length = len(log_returns.columns)
weight = RandWeights(length)
weight

In [ ]:
# Generating Portfolio Statistics
pf_stats = portfolio_stats(weight)

pf_return = pf_stats['return']
pf_risk = pf_stats['risk']
sharpe_ratio = pf_stats['sharpe']

In [ ]:
print(f'Portfolio Return    : {pf_return * 100:.2f} %')
print(f'Portfolio Risk      : {pf_risk * 100:.2f} %')
print(f'Portfolio Sharpe    : {sharpe_ratio * 100:.2f} %')

#### We will now run a monte carlo simulation to generate random portfolios. We will use the results of simulation to draw an efficient frontier

In [ ]:
def Monte_Carlo(iterations):
    portfolio_returns = []
    portfolio_risks = []
    for x in range (iterations):
        weight = RandWeights(length)
        pf_stats = portfolio_stats(weight)
        portfolio_returns.append(pf_stats['return'])
        portfolio_risks.append(pf_stats['risk'])
        
    portfolio_returns = np.array(portfolio_returns)
    portfolio_risks = np.array(portfolio_risks)
    return portfolio_returns, portfolio_risks

    

In [ ]:
portfolio_returns, portfolio_risks = Monte_Carlo(100000)
sharpe = portfolio_returns / portfolio_risks
max_sr_ret = portfolio_returns[sharpe.argmax()] # return corresponding to maximum sharpe ratio
max_sr_vol = portfolio_risks[sharpe.argmax()] # risk corresponding to maximum sharpe ratio
plt.figure(figsize=(18,10))
plt.scatter(portfolio_risks, portfolio_returns, c=sharpe, cmap='viridis')
plt.colorbar(label='Sharpe Ratio')
plt.xlabel('Risk')
plt.ylabel('Return')
plt.scatter(max_sr_vol, max_sr_ret,c='red', s=50) # red dot
plt.show()

- The above plot shows comparison of all portfolio combinations generated in Mone Carlo Simulation in terms of their risk and return. The red dot corresponds to the portfolio having the highest sharpe ratio amoung the generated portfolios. ( This portfolio may not be the one with highest sharpe ratio as we are plotting random portfolios. It is just the portfolio with highest sharpe ratio amoung all the randomly generated portfolios)
- We will now try to generate optimized portffolios subject to various conditions
- This hyperbolic plot is called 'Markowitz's Bullet'

#### Using Optimization to find portfolio with max sharpe ratio
- The below function returns the weights array cooresponding to the portfolio with the highest Sharpe Ratio
- We are using Scipy.optimize.minimize. We are trying to minimize negative Sharpe Ratio (which is same as maximising the sharpe ratio)
- The constraint for optimization is -> Sum of all the weights has to be 1, and all the weights are bounded between 0 and 1

In [ ]:
def OptimizationWithSharpeRatio():

    def FindNegSharpe(weight):
        return (-1)*portfolio_stats(weight)['sharpe']

    res = minimize(
          FindNegSharpe,
          RandWeights(length),
          method = 'SLSQP',
          constraints=[
            {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.},
          ],
          bounds=[(0., 1.) for i in range(length)]
        )
    
    return res


In [ ]:
OptimizationWithSharpeRatio()

- The optimization is successful. 
- The required weights are in the key x

#### Using Optimisation to find portfolio that has minimum risk for a given expected return

- Sometimes, the investors want to have a portfolio with a fixed targert return. 
- They want to find portfolio that would provide that return with minimum risk involved

In [ ]:
def OptimizationForAGivenReturn(target_return):
    
    def fun(weight):
        pf_stats = portfolio_stats(weight)
        _risk = pf_stats['risk']
        return _risk
    
    res = minimize(
      fun,
      RandWeights(length),
      method = 'SLSQP',
      constraints=[{'type':'eq','fun': lambda x: portfolio_stats(x)['return']-target_return},
                   {'type':'eq','fun': lambda x: np.sum(x)-1}],
      bounds=[(0., 1.) for i in range(length)]
    )
    
    return res


In [ ]:
OptimizationForAGivenReturn(0.4)

- For a return of 40%, we can find the optimal portfolio corresponding to the weights generated above

#### Finding portfolio that provide the minimum risk

In [ ]:
def OptimizingWithMinRisk():
    
    def fun(weight):
        pf_stats = portfolio_stats(weight)
        _risk = pf_stats['risk']
        return _risk
        
    
    res = minimize(
      fun,
      RandWeights(length),
      method = 'SLSQP',
      constraints=[
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.},
      ],
      bounds=[(0., 1.) for i in range(length)]
    )

    return res

### Plotting the efficient Frontier

- The efficient frontier is the set of optimal portfolios that offer the highest expected return for a defined level of risk or the lowest risk for a given level of expected return. Portfolios that lie below the efficient frontier are sub-optimal because they do not provide enough return for the level of risk.

- We will plot the efficient frontier by taking the optimal portfolios for all possible returns

In [ ]:
target_returns = np.linspace(portfolio_returns.min(), portfolio_returns.max(),70)

minimal_risks = []
for target_return in target_returns:
    optimal = OptimizationForAGivenReturn(target_return)
    minimal_risks.append(optimal['fun'])

minimal_risks = np.array(minimal_risks)
print(minimal_risks)

In [ ]:
plt.figure(figsize=(18,10))

plt.scatter(portfolio_risks, portfolio_returns,
            c = ( portfolio_returns / portfolio_risks),
            marker = 'o')


# Plotting the efficient frontier
plt.scatter(minimal_risks,
            target_returns,
            c = (target_returns / minimal_risks),
            marker = 'x')


#Plotting the portfolio that has highest Sharpe Ratio
Optimal_weights_For_Highest_Sharpe_Ratio = OptimizationWithSharpeRatio().x

plt.plot(portfolio_stats(Optimal_weights_For_Highest_Sharpe_Ratio)['risk'],
         portfolio_stats(Optimal_weights_For_Highest_Sharpe_Ratio)['return'],
         'r*',
         markersize = 25.0, label = "Portfolio corresponding to max sharpe")


#Plotting the optimal portfolio that has lowest risk
Optimal_weights_For_Lowest_Risk = OptimizingWithMinRisk().x

plt.plot(portfolio_stats(Optimal_weights_For_Lowest_Risk)['risk'],
         portfolio_stats(Optimal_weights_For_Lowest_Risk)['return'],
         'g*',
         markersize = 25.0, label = "Portfolio corresponding to lowest risk")

#Plotting the optimal portfolio for 25% annual returns
Optimal_weights_for_twenty_five_percent_returns = OptimizationForAGivenReturn(0.25).x

plt.plot(portfolio_stats(Optimal_weights_for_twenty_five_percent_returns)['risk'],
         portfolio_stats(Optimal_weights_for_twenty_five_percent_returns)['return'],
         'b*',
         markersize = 25.0, label = "Optimal Portfolio corresponding to 25% return")

#Plotting the optimal portfolio for 35% annual returns
Optimal_weights_for_thirty_five_percent_returns = OptimizationForAGivenReturn(0.35).x

plt.plot(portfolio_stats(Optimal_weights_for_thirty_five_percent_returns)['risk'],
         portfolio_stats(Optimal_weights_for_thirty_five_percent_returns)['return'],
         'm*',
         markersize = 25.0, label = "Optimal Portfolio corresponding to 35% return")

plt.xlabel('Portfolio Risk',fontsize = 20)
plt.ylabel('Portfolio Return', fontsize = 20)
plt.legend(prop={'size': 10})
plt.colorbar(label='Sharpe ratio')

- The efficient frontier is different for different investors, depending upon the assets they are holding
- There is nothing like a single optimal portfolio. The efficient frontier is the collection of optimal portfolios.
- The investors can choose any optimal portfolio depending upon the risk they can take